# Computer Vision Lab – Image Loading, Pixel Operations, RGB→Gray Conversion, and Basic Projections

This Jupyter notebook is designed for an **introductory Computer Vision lab**.

By the end of this notebook, you will be able to:

- Load and display images using OpenCV and Matplotlib
- Inspect image properties (shape, data type, channels)
- Perform basic **pixel operations** (access, modify, brightness, contrast)
- Convert **RGB/BGR images to grayscale** (both manually and using OpenCV)
- Compute and visualize **basic projections** (horizontal and vertical intensity profiles)

> **Note:** Place an image file (e.g., `input.jpg`) in the same folder as this notebook, or update the `image_path` variable accordingly.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


## Helper Functions for Display

In [ ]:
def show_bgr(img, title="Image"):
    """Display a BGR (OpenCV) image as RGB using matplotlib."""
    if img is None:
        raise ValueError("Image is None. Check the image path.")
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(5, 5))
    plt.imshow(img_rgb)
    plt.title(title)
    plt.axis('off')
    plt.show()

def show_gray(img, title="Image"):
    """Display a grayscale image using matplotlib."""
    if img is None:
        raise ValueError("Image is None. Check the image path.")
    plt.figure(figsize=(5, 5))
    plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

## 1. Loading an Image

In Computer Vision, an image is usually represented as a **NumPy array**.

- Color image: shape is `(height, width, channels)`
- Grayscale image: shape is `(height, width)`
- OpenCV loads color images in **BGR order** by default (Blue, Green, Red).

In [ ]:
# Change this path if needed
image_path = 'input.jpg'

img = cv2.imread(image_path)

if img is None:
    raise FileNotFoundError(f"Could not read image from {image_path}. Place an image in this folder or update the path.")

print("Type:", type(img))
print("Shape (H, W, C):", img.shape)
print("Data type:", img.dtype)

show_bgr(img, "Original Color Image (BGR shown as RGB)")

## 2. Basic Pixel Operations

An image is just a matrix of pixel values. You can **read** and **modify** pixel values using standard NumPy indexing.

- Pixel at row `y`, column `x`: `img[y, x]`
- For color image: this returns 3 values (B, G, R)
- For grayscale: this returns a single intensity value

In [ ]:
# Choose a pixel coordinate
y, x = 50, 100  # row, column

pixel_bgr = img[y, x]
print(f"Pixel value at (y={y}, x={x}) [B, G, R]:", pixel_bgr)

# Modify this pixel to pure green (B=0, G=255, R=0)
img_modified = img.copy()
img_modified[y, x] = [0, 255, 0]

show_bgr(img_modified, f"Modified Image with Green Pixel at ({x}, {y})")

### 2.1 Region of Interest (ROI)

You can also select and modify a **region** (sub-image) using slicing:

`roi = img[y1:y2, x1:x2]`

Below, we select a rectangular region and draw a colored rectangle on it.

In [ ]:
# Define a region of interest (ROI)
y1, y2 = 50, 150
x1, x2 = 100, 250

img_roi_demo = img.copy()

# Draw a red rectangle on the image to show ROI
cv2.rectangle(img_roi_demo, (x1, y1), (x2, y2), (0, 0, 255), 2)  # BGR: Red

show_bgr(img_roi_demo, "Region of Interest (ROI) Marked with Red Rectangle")

roi = img[y1:y2, x1:x2]
show_bgr(roi, "Cropped ROI")

## 3. Pixel-Wise Intensity Operations (Brightness & Contrast)

Two common pixel-wise operations:

- **Brightness change:** add or subtract a constant from every pixel
- **Contrast change:** multiply all pixel values by a factor

Formally, for each pixel value `p`:

\begin{equation}
p' = \alpha p + \beta
\end{equation}

where:
- $\alpha$ controls **contrast** (e.g., 1.2 = increase, 0.8 = decrease)
- $\beta$ controls **brightness** (e.g., +30 = brighter, -30 = darker)

OpenCV provides `cv2.convertScaleAbs` to help with this.

In [ ]:
alpha = 1.2  # contrast factor
beta = 30    # brightness offset

bright_contrast = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

show_bgr(img, "Original Image")
show_bgr(bright_contrast, f"Adjusted Image (alpha={alpha}, beta={beta})")

## 4. RGB/BGR to Grayscale Conversion

A color image has three channels (B, G, R in OpenCV). To convert it to grayscale, we need to combine these channels into a **single intensity value** per pixel.

### 4.1 Using OpenCV's Built-In Function

OpenCV provides a direct method:

`gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)`

In [ ]:
gray_cv = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

print("Grayscale shape:", gray_cv.shape)
show_gray(gray_cv, "Grayscale Image (OpenCV)")

### 4.2 Manual Grayscale Conversion

A common formula for converting RGB (or BGR) to grayscale uses a **weighted sum** of channels:

\begin{equation}
Y = 0.299 R + 0.587 G + 0.114 B
\end{equation}

In BGR format (OpenCV), the order is `(B, G, R)`, so we apply the weights accordingly.

In [ ]:
# Extract B, G, R channels
B = img[:, :, 0].astype(np.float32)
G = img[:, :, 1].astype(np.float32)
R = img[:, :, 2].astype(np.float32)

# Weighted sum for grayscale
gray_manual = 0.114 * B + 0.587 * G + 0.299 * R
gray_manual = np.clip(gray_manual, 0, 255).astype(np.uint8)

show_gray(gray_manual, "Grayscale Image (Manual Conversion)")

### 4.3 Simple Average Grayscale (for Comparison)

Another simpler (but less perceptually accurate) method is to take the **average** of the channels:

\begin{equation}
Y = \frac{R + G + B}{3}
\end{equation}

This treats all channels equally.

In [ ]:
gray_avg = img.mean(axis=2).astype(np.uint8)

show_gray(gray_avg, "Grayscale Image (Simple Average)")

### 4.4 Comparison of Grayscale Methods

Let's compare the three grayscale images side by side:
- OpenCV built-in (`cv2.cvtColor`)
- Manual weighted sum
- Simple average

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(gray_cv, cmap='gray')
plt.title('OpenCV Gray')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(gray_manual, cmap='gray')
plt.title('Manual Weighted Gray')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(gray_avg, cmap='gray')
plt.title('Average Gray')
plt.axis('off')

plt.show()

## 5. Basic Projections (Intensity Profiles)

**Projections** in image processing often refer to summarizing an image along one axis, for example:

- **Horizontal projection:** sum (or mean) of pixel intensities in each row
- **Vertical projection:** sum (or mean) of pixel intensities in each column

These projections are useful for:
- Document analysis (detecting text lines)
- Object localization
- Rough segmentation and alignment

In [ ]:
# Use a grayscale image for projections
gray = gray_cv

# Horizontal projection: sum of intensities along each row
horizontal_proj = np.sum(gray, axis=1)

# Vertical projection: sum of intensities along each column
vertical_proj = np.sum(gray, axis=0)

print("Horizontal projection shape:", horizontal_proj.shape)
print("Vertical projection shape:", vertical_proj.shape)

### 5.1 Visualizing the Horizontal Projection

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(horizontal_proj)
plt.title('Horizontal Projection (Row-wise Sum of Intensities)')
plt.xlabel('Row index (y)')
plt.ylabel('Sum of intensities')
plt.grid(True)
plt.show()

### 5.2 Visualizing the Vertical Projection

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(vertical_proj)
plt.title('Vertical Projection (Column-wise Sum of Intensities)')
plt.xlabel('Column index (x)')
plt.ylabel('Sum of intensities')
plt.grid(True)
plt.show()

### 5.3 Overlay Projections for Intuition

To build intuition, we can visualize the projections next to the grayscale image.

In [ ]:
fig = plt.figure(figsize=(12, 6))

# Show grayscale image
ax1 = fig.add_subplot(1, 2, 1)
ax1.imshow(gray, cmap='gray')
ax1.set_title('Grayscale Image')
ax1.axis('off')

# Show both projections in one figure (stacked)
ax2 = fig.add_subplot(2, 2, 2)
ax2.plot(horizontal_proj, np.arange(len(horizontal_proj)))
ax2.invert_yaxis()
ax2.set_title('Horizontal Projection')
ax2.set_xlabel('Sum of intensities')
ax2.set_ylabel('Row index')

ax3 = fig.add_subplot(2, 2, 4)
ax3.plot(np.arange(len(vertical_proj)), vertical_proj)
ax3.set_title('Vertical Projection')
ax3.set_xlabel('Column index')
ax3.set_ylabel('Sum of intensities')

plt.tight_layout()
plt.show()

## 6. Summary

In this lab notebook, you have:

- Loaded an image using OpenCV
- Displayed images in color and grayscale using Matplotlib
- Performed basic **pixel operations**:
  - Accessing and modifying individual pixels
  - Selecting and displaying regions of interest (ROI)
  - Changing brightness and contrast
- Converted a color image from **BGR to grayscale**:
  - Using OpenCV (`cv2.cvtColor`)
  - Using a **manual weighted sum**
  - Using a simple channel average
- Computed and visualized **horizontal and vertical projections** of a grayscale image

### Suggested Exercises

1. Try different values of `alpha` and `beta` for contrast and brightness.
2. Write a function that normalizes the image to use full intensity range [0, 255].
3. Instead of `sum`, compute **mean** projections and compare results.
4. Apply projections only on a selected ROI (e.g., top half or central region).